# Redes Neuronales Recurrentes (RNR)/ Recurrent Neuronal Networks (RNN)

## Parte 1 - Preprocesado de los datos

In [1]:
#Importación de las librerías
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
# Importar el dataset de entrenamiento
dataset_train = pd.read_csv("Google_Stock_Price_Train.csv")
training_set = dataset_train.iloc[:, 1:2].values

In [5]:
# Escalado de las características
# Mejor usar la normalización en lugar de la estandarización

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range= (0,1)) # el más pequeño será 0 y el más grande 1. No hace falta porue es así por defecto
training_set_scaled = sc.fit_transform(training_set)                

In [7]:
# Crear una estructura de datos con 60 timesteps y 1 salida
# 60 timesteps-> en cada momento del tiempo la red neuronal será capaz de mirar 60 días atrás
X_train =[]
y_train =[]
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i,0]) # para cada día a partir de 60 miramos 60 días antes
    y_train.append(training_set_scaled[i, 0]) # y la etiqueta será ese día
X_train, y_train = np.array(X_train), np.array(y_train) # Para que sean objetos de numpy

In [9]:
# Redimensión de los datos
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# ha pasado de (1198, 60) a (1198, 60, 1) -> 1198 filas, 60 columnas y 1 dimensión # esto me lo tienen que explicar mejor

En Keras (y en la mayoría de librerías), una RNN / LSTM espera los datos de entrada con esta forma:


· n_muestras → el número de ejemplos de entrenamiento (aquí 1198).

· timesteps → cuántos pasos de tiempo mira la red (aquí 60 días).

· n_features → cuántas variables hay en cada día (en cada timestep).

En mi caso:

Cada día sólo tienes 1 feature: el precio de apertura (Open).

Por eso n_features = 1.


Si algún día meto más variables por día, por ejemplo:
Open, High, Low, Volume, tendrías:

4 features por día → n_features = 4.

La forma sería (1198, 60, 4).




## Parte 2 - Construcción de la RNR

## Parte 3 - Ajustar las predicciones y visualizar los resultados